In [1]:
# Load model directly
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
torch.manual_seed(0)


/nfshomes/astein0/.pyenv/versions/long_context/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!nvidia-smi

Tue Jul 23 15:50:51 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A4000               Off |   00000000:C1:00.0 Off |                  Off |
| 41%   24C    P8             13W /  140W |       1MiB /  16376MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
# path = 'openbmb/MiniCPM-2B-dpo-bf16'
path = "openbmb/MiniCPM-2B-128k"
CPM_tokenizer = AutoTokenizer.from_pretrained(path)
CPM_model = AutoModelForCausalLM.from_pretrained(path, device_map="auto", trust_remote_code=True)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf", device_map="auto", torch_dtype=torch.bfloat16)

Loading checkpoint shards: 100%|██████████| 2/2 [00:12<00:00,  6.09s/it]


In [134]:
# Encode the input text
def create_prompt(pgn_text):
    # input_text = f"I am going to give you a PGN that depicts a chess game and I am going \
    # to ask you questions about the state of the game at different times.\n\
    # As a reminder, a PGN is a chess game format that contains header information then the game moves as the last line.\n\
    # As a test run, lets start by only asking one question:\nWho won the game?\n\
    # Just output the single word corresponding to the result of the game.\
    # For example, if white won the game output 'white', if black won the game output 'black' and if its a tie output 'tie'.\n\
    # Here is the PGN:\n{pgn_text}"
    input_text = f"{pgn_text}\nWhat was the result of the game?\n\
    Your entire response should only be the result of the game (and no other characters or response words):\
    For example, if white won the game, the entire response should be '1-0', if black won the game the response should be '0-1' and if its a tie the response should be '1/2-1/2'.\n\
    "

    # inputs = tokenizer(input_text, return_tensors="pt")

    return input_text

assistent_response = "The result of the game is: "

# prompt = create_prompt(all_pgns[0]["text"])
# prompt

In [113]:
def create_prompt(pgn_text):
    input_text = f"{pgn_text}\nHow many pieces are on the queenside of the board after 10 full moves (10 for white and 10 for black)?\n\
    Respond only with an integer representing your answer."

    return input_text
    
assistent_response = "After 10 moves the number of pieces on the queen side of the board is: "


In [106]:
system_prompt = "You are a chess bot charged with helping answer questions about chess games.  \
Users are going to give you a PGN that depicts a chess game and then \
ask you questions about the game. As a reminder, a PGN is a chess game format that \
contains header information then the game moves as the last line.\n"


In [135]:
def create_message(system_prompt, user_prompt, assistent_response):
    messages = [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': user_prompt},
        {'role': 'assistant', 'content': assistent_response},
    ]
    return messages

In [ ]:
# tokenizer.chat_template
# decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
mod = CPM_model
tok = CPM_tokenizer

mod = model
tok = tokenizer


total = 0
correct = 0
for pgn in all_pgns:
    user_prompt = create_prompt(pgn["text"])
    messages = create_message(system_prompt, user_prompt, assistent_response)
    formatted_text = tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tok(formatted_text, return_tensors='pt', add_special_tokens=False)
    inputs = {key: tensor.to(mod.device) for key, tensor in inputs.items()}
    outputs = mod.generate(**inputs, max_new_tokens=64)
    decoded = tok.decode(outputs[0][inputs['input_ids'].size(1):], skip_special_tokens=True)
    print(tok.decode(outputs[0], skip_special_tokens=True))

    # did it get correct winner?
    ref = pgn["result"]
    dec = decoded.strip().split()[0]
    
    # did it count number of pieces correct on board after 10 moves?
    # ref = pgn["queenside_10"]
    # decoded = decoded.strip()
    
    pgn["output"] = decoded
    correct += 1 if dec == ref else 0
    total += 1
    if total % 100 == 0:
        print(f"total: {total}, correct: {correct}")
print(correct)